## Optimize CNN model using Keras Tuner

In [1]:
!pip install keras-tuner==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 2.5 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation

In [3]:
print(tf.__version__)

2.13.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [10]:
print(train_labels.shape)
print(test_labels.shape)

(60000,)
(10000,)


In [11]:
#Method#2
def build_model(hp):  # random search passes this hyperparameter() object
    model = keras.models.Sequential()

    model.add(Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3), input_shape=(28,28,1)))

    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    for i in range(hp.Int('n_connections', 1, 4)):
        model.add(Dense(hp.Choice(f'n_nodes',
                                  values=[128, 256, 512, 1024])))
        model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation("softmax"))

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [12]:
from kerastuner import RandomSearch

In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=2,directory='output_mnist',project_name="mnist_Fashion")

In [14]:
tuner_search.search(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=2
)

Epoch 1/2
1688/1688 [==============================] - 24s 8ms/step - loss: 0.5829 - accuracy: 0.7814 - val_loss: 0.4303 - val_accuracy: 0.8405
Epoch 2/2
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3748 - accuracy: 0.8623 - val_loss: 0.3642 - val_accuracy: 0.8620


Epoch 1/2
1688/1688 [==============================] - 13s 7ms/step - loss: 0.4280 - accuracy: 0.8415 - val_loss: 0.3674 - val_accuracy: 0.8607
Epoch 2/2
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2850 - accuracy: 0.8935 - val_loss: 0.2848 - val_accuracy: 0.8922


In [15]:
tuner_search.results_summary()

In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 224)       2240      
                                                                 
 activation (Activation)     (None, 26, 26, 224)       0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 224)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        129088    
                                                                 
 activation_1 (Activation)   (None, 11, 11, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 192)         110784    
                                                        

In [18]:
model.fit(train_images, train_labels, epochs=3, validation_split=0.1, initial_epoch=2)

Epoch 3/3
1688/1688 [==============================] - 13s 6ms/step - loss: 0.2347 - accuracy: 0.9121 - val_loss: 0.2580 - val_accuracy: 0.9017


In [19]:
# Evaluate the best model.
loss, accuracy = model.evaluate(test_images, test_labels)
print('loss:', loss)
print('accuracy:', accuracy)

313/313 [==============================] - 1s 4ms/step - loss: 0.2659 - accuracy: 0.9062
loss: 0.2658640146255493
accuracy: 0.9061999917030334


In [33]:
# save model
model.save('final_model1.h5')

In [34]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
 # load the image
 img = load_img(filename, grayscale=True, target_size=(28, 28))
 # convert to array
 img = img_to_array(img)
 # reshape into a single sample with 1 channel
 img = img.reshape(1, 28, 28, 1)
 # prepare pixel data
 img = img.astype('float32')
 img = img / 255.0
 return img

# load an image and predict the class
def run_example():
 # load the image
 img = load_image('sample_image.png')
 # load model
 model = load_model('final_model1.h5')
 # predict the class
 result = model.predict(img)
 print(result[0])
 classes_x=np.argmax(result,axis=1)
 print(classes_x)

# entry point, run the example
run_example()

1/1 [==============================] - 0s 185ms/step
[1.1226135e-04 2.9274995e-08 9.9839419e-01 4.2619720e-07 7.0592162e-04
 3.0712706e-09 7.8643102e-04 8.0477927e-09 6.7569789e-07 5.5377409e-08]
[2]
